In [1]:
import fsspec
import s3fs
import numpy as np
import imageio
import datetime
from dateutil import tz


In [2]:
# Check for files from today
# This makes a "naive" timestamp
#tsNow = datetime.datetime.utcnow().timestamp()
# This makes an "aware" timestamp
tsNow = datetime.datetime.now(tz=datetime.timezone.utc).timestamp()

print("Current UTC time stamp: ",tsNow)
dayn = str(int(tsNow/100000.))
print("Day number: ",dayn)
today = datetime.date.today()
print("Today is:",today)

Current UTC time stamp:  1643326340.416366
Day number:  16433
Today is: 2022-01-27


In [25]:
def unix2dts(unixnumber, timezone='eastern'):
    """
    Get local time from unix number

    Input:
        unixnumber - string containing unix time (aka epoch)
    Returns:
        date_time_string, date_time_object in utc

    TODO: not sure why this returns the correct value without specifying that input time zone is eastern
    """
    if timezone.lower() == 'eastern':
        tzone = tz.gettz('America/New_York')
    elif timezone.lower() == 'utc':
        tzone = tz.gettz('UTC')

    # images other than "snaps" end in 1, 2,...but these are not part of the time stamp.
    # replace with zero
    ts = int( unixnumber[:-1]+'0')
    date_time_obj =  datetime.datetime.utcfromtimestamp(ts)
    date_time_str = date_time_obj.strftime('%Y-%m-%d %H:%M:%S')
    return date_time_str, date_time_obj

unix2dts('1642692600')

('2022-01-20 15:30:00', datetime.datetime(2022, 1, 20, 15, 30))

In [3]:
def dayn( timestamp ):
    daynf = np.floor(tsNow/10000.)
    daync = np.ceil(tsNow/10000.)
    print(daynf, daync)
    return daynf, daync 

tsStart = datetime.datetime.strptime('2020-11-01 12:00','%Y-%m-%d %H:%M')
tsStart.replace(tzinfo=datetime.timezone.utc)
print(tsStart, tsStart.timestamp(), dayn(tsStart.timestamp()))
tsEnd = datetime.datetime.strptime('2020-11-02 12:00','%Y-%m-%d %H:%M')
tsEnd.replace(tzinfo=datetime.timezone.utc)
print(tsEnd, tsEnd.timestamp(), dayn(tsEnd.timestamp()))
print(tsEnd.timestamp()-tsStart.timestamp())
print(str(int(tsStart.timestamp())+1))

164332.0 164333.0
2020-11-01 12:00:00 1604250000.0 (164332.0, 164333.0)
164332.0 164333.0
2020-11-02 12:00:00 1604336400.0 (164332.0, 164333.0)
86400.0
1604250001


In [22]:
# Get the timestamp for the day you want images
datestr = '2022-01-26'
ts = datetime.datetime.strptime(datestr,'%Y-%m-%d')
print(ts.timestamp())
timestr = str(ts.timestamp())[0:6]
print(timestr)

1643173200.0
164317


In [23]:
dayn(1642692600)

164332.0 164333.0


(164332.0, 164333.0)

In [20]:
fs = fsspec.filesystem('s3')
globstring = 'cmgp-coastcam/cameras/caco-01/products/'+timestr+'*.c?.timex.jpg'
recent_list=fs.glob(globstring)
print(recent_list)

In [21]:
print(recent_list)

[]


### strategy
Not sure what the best strategy is. I think we tend to miss some c2 files, so maybe the best plan it find the last c2 file and the corresponding c1 file.

In [18]:
ss = recent_list[-1].split('.')
print(ss)
c1fn = ss[0]+'.c1.'+ss[2]+'.'+ss[3]
print(c1fn)

['cmgp-coastcam/cameras/caco-01/products/1602880201', 'c2', 'timex', 'jpg']
cmgp-coastcam/cameras/caco-01/products/1602880201.c1.timex.jpg


In [20]:
# is there a corresponding c1 file?
for e in recent_list:
    if e == c1fn:
        print(e)

cmgp-coastcam/cameras/caco-01/products/1602880201.c1.timex.jpg


In [21]:

with fs.open(c1fn) as f:
    im = imageio.imread(f)
    
print(im.shape)

(2048, 2448, 3)


In [4]:
recent_list=fs.glob('cmgp-coastcam/cameras/caco-01/latest/*')
print(recent_list)

['cmgp-coastcam/cameras/caco-01/latest/1594377000.c1.snap.jpg', 'cmgp-coastcam/cameras/caco-01/latest/1594377000.c2.snap.jpg', 'cmgp-coastcam/cameras/caco-01/latest/1594377001.c1.timex.jpg', 'cmgp-coastcam/cameras/caco-01/latest/1594377001.c2.timex.jpg', 'cmgp-coastcam/cameras/caco-01/latest/1594377002.c1.var.jpg', 'cmgp-coastcam/cameras/caco-01/latest/1594377002.c2.var.jpg', 'cmgp-coastcam/cameras/caco-01/latest/1594377003.c1.bright.jpg', 'cmgp-coastcam/cameras/caco-01/latest/1594377003.c2.bright.jpg', 'cmgp-coastcam/cameras/caco-01/latest/1594377004.c1.dark.jpg', 'cmgp-coastcam/cameras/caco-01/latest/1594377004.c2.dark.jpg', 'cmgp-coastcam/cameras/caco-01/latest/1594377005.c1.rundark.jpg', 'cmgp-coastcam/cameras/caco-01/latest/1594377005.c2.rundark.jpg', 'cmgp-coastcam/cameras/caco-01/latest/1594378800.c1.snap.jpg', 'cmgp-coastcam/cameras/caco-01/latest/1594378800.c2.snap.jpg', 'cmgp-coastcam/cameras/caco-01/latest/1594378801.c1.timex.jpg', 'cmgp-coastcam/cameras/caco-01/latest/15943

In [5]:
# test writing to the public bucket
fs2 = fsspec.filesystem('s3', profile='default')

with fs2.open('s3://cmgp-sfm-public-read-bucket/csherwood/foo.txt','w') as fo:
    fo.write('test file text')

    